In [2]:
# The usual fare
# Importing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from astropy.io import fits
import matplotlib.cm as cm
import os 
import glob 
import astropy
import scipy.signal
from astropy.stats import sigma_clip
import time
from astropy import stats
from PIL import Image
import shift_methods as sm
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

In [80]:
def CustomBias(filelist,masterbias, **kwargs):
    '''
    CustomBias
    =======================
    Makes a custom bias frame using an image's overscan and performs a bias subtraction using it.
    
    input
    =======================
    filelist   : list of files you want to do custom bias subtract
    masterbias: the normalized master bias data
    
    kwargs
    =======================
    printstuff : if True, it will print the name of each bias
                subtracted file as it is generated
    
    returns
    =======================
    n/a
    
    files written
    =======================
    bias subtracted frame

    '''
    print('Custom Biases started')
    print('*'*25)
    for i in range(len(filelist)):
        header = fits.getheader(filelist[i])
        im = fits.getdata(filelist[i])
        new_name = 'b_'+filelist[i][6:-7]+'.fits'
        if im.shape == (4150,4150):
            #im_name = file[6:-7]
            overscan = np.mean(im[:,4100:4140])
            custom_bias = overscan * masterbias
            bias_subtracted = im - custom_bias
            bias_sub_overscan = bias_subtracted[:4096,:4096]
            fits.writeto(new_name,bias_sub_overscan, header,overwrite=True)
            if 'printstuff' in kwargs:        
                print(new_name,'written.')
                print('*'*25)
    print('Custom Biases done')
            
def norm_combine_flats(filelist):
    '''
    norm_combine_flats
    =======================
    This normalizes the flat field images
    
    inputs
    =======================
    filelist:: List of flats you need to combine in master flat
    
    returns
    =======================
    med_frame: median combined normalized frame
    
    files written
    =======================
    n/a
    '''
    print('Starting to norm_combine_flats')
    print('*'*25)
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        norm_im =  im/np.median(im)# finish new line here to normalize flats
        fits_stack[:,:,ii] = norm_im
    med_frame = np.median(fits_stack, axis=2)
    print('Done norm_combine_flats')
    return med_frame



def FileMover(filelist,foldername):
    '''
    FileMover
    =======================
    More general version of filesorter.

    inputs
    =======================
    filelist  : The list of files you want to move to a new folder
    foldername: The name of the folder you want to move the files to

    returns
    =======================
    n/a

    '''
    for file in filelist:
        if os.path.exists(file):
            pass
        else:
            print(file + "does not exist or has already been moved")
            return
        if os.path.exists(foldername):
            pass
        else:
            print("Making new directory:" + foldername)
            os.mkdir(foldername)
        destination = foldername + '/'
        print("Moving " + file + " to: ./" + destination + file)
        os.rename(file, destination + file)  
    print('Done moving files.')
            
def imageshifter(imlist):
    '''
    imageshifter
    =======================
    Shifts/aligns images
    
    inputs
    =======================
    imlist: list of images you want to align to each other
    
    returns
    =======================
    n/a
    
    files written
    =======================
    shifted image
    '''
    imlist=imlist
    imlist.sort()
    # i=0
    # worklist= imlist[i:i+3]
    # for i in range(len(imlist)):
    im1,hdr1 = fits.getdata(imlist[0],header=True)
    #     # print(imlist[0])
    xshifts={}
    yshifts={}
    shift_image={}

    for index,filename in enumerate(imlist):
        header=fits.getheader(filename)
        imstack = np.zeros([im1.shape[0],im1.shape[1],len(imlist)])
        im = fits.getdata(filename)
        xshifts[index],yshifts[index]=sm.cross_image(im1,im,boxsize=2000)
        shift_image[index] = sm.shift_image(im,xshifts[index],yshifts[index])
        name = 's_'+filename
        fits.writeto(name,shift_image[index],header,overwrite=True)
        print("Shift for image",name,"is",xshifts[index],yshifts[index])
        print('FITS file written:',name)
        print('*'*25)
    print('Shifting done!')
     
    
def mediancombine(filelist):
    '''
    median combine
    =======================
    Takes in a list of frames and does a medain combination on them.
    Used for biases, master flats, and basic image combination.
    
    inputs
    =======================
    filelist : The list of files you want to median combine
    
    returns
    =======================
    med_frame: The median combined frame
    '''
    print('Starting to median combine')
    print('*'*25)
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        fits_stack[:,:,ii] = im
    med_frame = np.median(fits_stack, axis=2)
    print('Done median combining')
    return med_frame


def filesorter(filename, foldername, fitskeyword_to_check, keyword,printstuff):
    '''
    filesorter
    =======================
    more specific than FileMover. Will cycle through images to see
    if a specific keyword matches a specific value. requires a
    for loop looping over a list of files to work
    
    inputs
    =======================
    filename : name of file you want to check to see if it should be sorted
    foldername : name of folder you want to move the sorted files to
    fitskeyword_to_check : value you want to check
    keyword              : fits keyword you want to check for the value specified
    
    kwargs
    =======================
    printstuff : if True, will print information about files being moved
    
    
    '''
    print('Starting to filesort')
    print('*'*25)
    if os.path.exists(filename):
        pass
    else:
        print(filename + " does not exist or has already been moved.")
        return
    
    header = fits.getheader(filename)
    fits_type = header[keyword]
    
    if os.path.exists(foldername):
        pass
    else:
        print("Making new directory: " + foldername)
        os.mkdir(foldername)
        
    if fits_type == fitskeyword_to_check:
        destination = foldername + '/'
        if 'printstuff' in kwargs:
            print("Moving " + filename + " to: ./" + destination + filename)
        os.rename(filename, destination + filename)  
    print('Done filesorting')
    return

def ministack(imlist,obj,filt,**kwargs):
    '''
    ministack
    =======================
    stacks science images
    
    inputs
    =======================
    imlist    : list of images to work with
    obj       : name of object in frames
    filt      : name of filter of frames
    
    kwargs
    =======================
    num       : how many images you want each stack to be
    If no kwargs specified, it will stack all the images in the list.
    '''

    if 'num' in kwargs:
        imperstack = kwargs['num']
#         print(imperstack)
        im1 = fits.getdata(imlist[0])
        i=0
        worklist=imlist[i:i+imperstack]
        print('Starting Ministacks')
        print('*'*25)
        for i in range(len(imlist)):
            if len(worklist)>=imperstack: 
                num = str(i)
    #             print(worklist[len(worklist)//2])
                header = fits.getheader(worklist[len(worklist)//2])
                medianimage = mediancombine(worklist)
                header['NUMSTACK'] = imperstack
                print('Stacking images:')
                print(worklist)
                fits.writeto(num+'_'+obj+'_'+filt+'_stack.fits',medianimage,header,overwrite=True)
                print('Wrote FITS File:',num+'_'+obj+'_'+filt+'_stack.fits')
                print('*'*25)

            else:
                break

            i+=i*i+imperstack
            worklist=imlist[i:i+imperstack]
    else:
        header = fits.getheader(imlist[len(imlist)//2])
        header['NUMSTACK']=len(imlist)
        medianimage = mediancombine(imlist)
        fits.writeto(obj+'_'+filt+'_Fullstack.fits',medianimage,header,overwrite=True)
        print('Wrote FITS File:',obj+'_'+filt+'_Fullstack.fits')
        print('*'*25)
    print('Done Ministacking')

In [16]:
target = 'HLTau'
targdec = '18:14:25'
targra = '4:32:3.0'
filtnum = '202'
filt='HaOff'
# imagetoalignto = 'E:/20190120/c8503t0205o00.fits'
masterpath = 'E:/20190122'
biaspath = 'E:/20190121/NormMasterBias.fits'
flatpath = masterpath + '/HaOff/flats/HaOffFlat.fits'

In [17]:
masterbias = fits.getdata(biaspath)
masterflat = fits.getdata(flatpath)

In [37]:
os.chdir(masterpath+'/'+filt)
ref_position = SkyCoord(targra,targdec, unit=(u.hourangle, u.deg))
targims = []
allims=glob.glob('c*.fits')
for im in allims:
    ra = fits.getheader(im)['RASTRNG']
    dec = fits.getheader(im)['DECSTRNG']
    image_position = SkyCoord(ra,dec,unit=(u.hourangle,u.deg))
    sep = ref_position.separation(image_position)
    if sep.arcsecond < 60:
          targims.append(im)
FileMover(targims,target)

Making new directory:HLTau
Moving c8505t0113o00.fits to: ./HLTau/c8505t0113o00.fits
Moving c8505t0114o00.fits to: ./HLTau/c8505t0114o00.fits
Moving c8505t0115o00.fits to: ./HLTau/c8505t0115o00.fits
Moving c8505t0157o00.fits to: ./HLTau/c8505t0157o00.fits
Moving c8505t0158o00.fits to: ./HLTau/c8505t0158o00.fits
Moving c8505t0159o00.fits to: ./HLTau/c8505t0159o00.fits


In [38]:
os.chdir(masterpath+'/'+filt+'/'+target)
allims = glob.glob('c*.fits')
CustomBias(allims,masterbias)
b_sci = glob.glob('b_*.fits')

b_0113o.fits written.
b_0114o.fits written.
b_0115o.fits written.
b_0157o.fits written.
b_0158o.fits written.
b_0159o.fits written.


In [19]:
os.chdir(masterpath+'/'+filt+'/'+target)
b_sci = glob.glob('b_*.fits')

for im in b_sci:
    data = fits.getdata(im)
    header = fits.getheader(im)
    flat_fielded = data / masterflat
    name = 'f_' + im
    fits.writeto(name,flat_fielded,header,overwrite=True)
    print(name + ' has been flatfielded')
f_sci = glob.glob('f_b_*.fits')
imageshifter(f_sci)
# shifted = glob.glob('s_f_b_*.fits')
# shifted.sort()
# ministack(shifted,target,filt)

f_b_0113o.fits has been flatfielded
f_b_0114o.fits has been flatfielded
f_b_0115o.fits has been flatfielded
f_b_0157o.fits has been flatfielded
f_b_0158o.fits has been flatfielded
f_b_0159o.fits has been flatfielded
Shift for image s_f_b_0113o.fits is 0.0 0.0
FITS file written: s_f_b_0113o.fits
*************************
Shift for image s_f_b_0114o.fits is -2.0128266268649213 3.5113160452643797
FITS file written: s_f_b_0114o.fits
*************************
Shift for image s_f_b_0115o.fits is 0.616507160043966 -0.46429352999803086
FITS file written: s_f_b_0115o.fits
*************************
Shift for image s_f_b_0157o.fits is -6.238255729980665 23.513587302199994
FITS file written: s_f_b_0157o.fits
*************************
Shift for image s_f_b_0158o.fits is 0.058060422870539696 -0.21290641529287768
FITS file written: s_f_b_0158o.fits
*************************
Shift for image s_f_b_0159o.fits is -0.2919447087039089 0.3178456750188161
FITS file written: s_f_b_0159o.fits
*****************

In [67]:
ministack(im,target,filt,num=3)

Starting Ministacks
*************************
Stacking images:
['s_f_b_0113o.fits', 's_f_b_0114o.fits', 's_f_b_0115o.fits']
Wrote FITS File: 0_HLTau_HaOff_stack.fits
*************************
Stacking images:
['s_f_b_0157o.fits', 's_f_b_0158o.fits', 's_f_b_0159o.fits']
Wrote FITS File: 1_HLTau_HaOff_stack.fits
*************************
Done Ministacking
